In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire
import wrangle

In [2]:
df = acquire.get_zillow_data()

ValueError: unsupported format character ''' (0x27) at index 323

Through domain research and choosing features that have enough data (low null values, less than 20%), I've determined these columns to be the most practical to utilize in the data science pipeline for the purpose of predicting tax assessed home values through regression modeling. 

In [ ]:
df.info()

In [ ]:
df.isna().sum()

To not lose too much data, I'm looking at columns with many null values and how many null values they share with other columns

In [ ]:
df['calculatedfinishedsquarefeet'].isna().sum() & df['lotsizesquarefeet'].isna().sum() & df['yearbuilt'].isna().sum()

square feet, lot size, and year built share a lot of null rows. This is good news when considering dropping columns with null values.

In [ ]:
df['lotsizesquarefeet'].isna().sum() & df['taxamount'].isna().sum(), df['calculatedfinishedsquarefeet'].isna().sum() & df['taxamount'].isna().sum()

most of tax amount nulls are also square feet nulls so again, feel better about dropping these records

In [ ]:
(df.isnull().sum() / len(df)).round(3) 
#this shows the percentage of null values per column rounded to the 3rd decimal place

we can see that all columns have less than 1% null (missing) values. This is favorable to dropping nulls

In [ ]:
df.isna().mean().sum()
#shows the percentage of missing values in the entire dataframe. 

If all nulls are dropped, we will lose 1.7% of the data. With this low percentage and seeing how most records with nulls have nulls across other columns, I determine dropping records with nulls is the most appropriate solution

In [ ]:
df = wrangle.wrangle_zillow(df)

In [ ]:
df

In [ ]:
df = wrangle.wrangle_locs(df)

In [ ]:
df.drop(columns= 'Unnamed: 0', inplace = True)

In [ ]:
df

In [ ]:
df.value.mean()

seeing that tax assessed value mean is very different based on the assessment year, I am dropping 2014 and 2015 assessment year records from the dataframe for consistency

In [ ]:
#df.assessmentyear.value_counts()

Now that all records have the same assessment year (2016), I can drop this column

In [ ]:
#df.drop(columns = ['assessmentyear'], inplace=True)

In [ ]:
df

In [ ]:
# df.rename(columns={'landtaxvaluedollarcnt': 'land_value', 'lotsizesquarefeet': 'lot_size', 'square_feet': 'home_size'}, inplace=True)

In [ ]:
# df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
# df

In [ ]:
# df['land_value'] = df.land_value.astype(int)

In [ ]:
# df['lot_size'] = df.lot_size.astype(int)

### Univariate Exploration

In [ ]:
df.bedrooms.value_counts().sort_values()

In [ ]:
df.bathrooms.value_counts().sort_values()

In [ ]:
# put this in wrangle function 

df = df[df.lot_size > 500]
df = df[df.home_size > 500]
df = df[df.value > 10000]

In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
print()
printmd("**Zillow Data (Min, Max, Average)**")
print("--------------------------------")
printmd("**Tax Assessed Value of Home**")
printmd('*Maximum Tax Assessed Value: {:,}*'
     .format(df['value'].max()))
printmd('*Minimum Tax Assessed Value: {:,}*'
     .format(df['value'].min()))
printmd('*Average Tax Assessed Value: {:,}*'
     .format(round(df['value'].mean())))
print("--------------------------------")
printmd("**Tax Amount of Home**")
printmd('*Maximum Tax Amount of Home: {:,}*'
     .format(round(df['tax'].max())))
printmd('*Minimum Tax Amount of Home: {:,}*'
     .format(df['tax'].min()))
printmd('*Average Tax Amount of Home: {:,}*'
     .format(round(df['tax'].mean())))
print("--------------------------------")
printmd("**Home Size in Square Feet**")
printmd('*Maximum Home Size: {:,} square feet*'
     .format(df['home_size'].max()))
printmd('*Minimum Home Size: {:,} square feet*'
     .format(df['home_size'].min()))
printmd('*Average Home Size: {:,} square feet*'
     .format(round(df['home_size'].mean())))
print("--------------------------------")
printmd("**Lot Size in Square Feet**")
printmd('*Maximum Lot Size: {:,} square feet*'
     .format(df['lot_size'].max()))
printmd('*Minimum Lot Size: {:,} square feet*'
     .format(df['lot_size'].min()))
printmd('*Average Lot Size: {:,} square feet*'
     .format(round(df['lot_size'].mean())))
print("--------------------------------")
printmd("**Tax Assessed Land Value**")
printmd('*Maximum Land Value: {:,}*'
     .format(df['land_value'].max()))
printmd('*Minimum Land Value: {:,}*'
     .format(df['land_value'].min()))
printmd('*Average Land Value: {:,}*'
     .format(round(df['land_value'].mean())))

In [ ]:
plt.figure(figsize=(10, 4))
sns.set_palette('Set2', 8)
plt.title('bedroom count')
sns.countplot(x = 'bedrooms', data= df)

In [ ]:
plt.figure(figsize=(14, 5))
sns.set_palette('Set2', 20)
sns.countplot(x='bathrooms', data=df)
plt.title('bathroom count')

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr

from math import sqrt
from scipy import stats

In [ ]:
# make this a function
train_and_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_and_validate, test_size=.3, random_state=123)

In [ ]:
train.info()

In [ ]:
orange_train = train.copy()

In [ ]:
ventura_train = train.copy()

In [ ]:
la_train = train.copy()

In [ ]:
orange_train = orange_train[orange_train.county == 'orange']

In [ ]:
orange_train

In [ ]:
ventura_train = ventura_train[ventura_train.county == 'ventura']

In [ ]:
la_train = la_train[la_train.county == 'los_angeles']

In [ ]:
ventura_train.to_csv("ventura_locations.csv")
la_train.to_csv("la_locations.csv")
orange_train.to_csv("orange_locations.csv")

In [ ]:
ventura_train.info()

In [ ]:
ventura_train['latitude'] = ventura_train.latitude.astype(str)

In [ ]:
ventura_train['latitude'].dtype

In [ ]:
ventura_train['latitude'] = np.where(ventura_train['latitude']==' ',0, ventura_train['latitude']).astype(float)

In [ ]:
ventura_train['longitude'] = ventura_train.longitude.astype(str)

In [ ]:
ventura_train['longitude'] = ventura_train['longitude'].str.rstrip('.0') 

In [ ]:
ventura_train['longitude']

In [ ]:
ventura_train['longitude'] = np.where(ventura_train['longitude']==' ',0, ventura_train['longitude']).astype(float)

In [ ]:
import geopy

In [ ]:
def get_zipcode(ventura_train, geolocator, lat_field, lon_field):
    location = geolocator.reverse((ventura_train['latitude'], ventura_train['longitude']))
    return location.raw['address']['postcode']

geolocator = geopy.Nominatim(user_agent='mlshiben')

In [ ]:
ventura_train.get('latitude')

In [ ]:
import geocoder

In [ ]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=50)

In [ ]:
def geo_rev(ventura_train):
    g = geocoder.osm([ventura_train.latitude, ventura_train.longitude], method='reverse').json
    if g:
        return g.get('postal')
    else:
        return 'no postal'

ventura_zips = ventura_train[['latitude', 'longitude']].apply(geo_rev, axis=1)


In [ ]:
g = geocoder.osm(ventura_train['latitude'], ventura_train['longitude']), method='reverse')
g.json['postal']

In [ ]:
zipcodes = ventura_train.apply(get_zipcode, geolocator=geolocator, lat_field ='latitude', lon_field = 'longitude')